# 目的
VGG16モデルを用いた転移学習をFXTFのデータに適用する。   
[VGG16のFine-tuningによる17種類の花の分類 - 人工知能に関する断創録](http://aidiary.hatenablog.com/entry/20170131/1485864665)の次のコードを参考にする：   
```
# VGG16モデルと学習済み重みをロード
# Fully-connected層（FC）はいらないのでinclude_top=False）
input_tensor = Input(shape=(img_rows, img_cols, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

# FC層を構築
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(nb_classes, activation='softmax'))

# VGG16とFCを接続
model = Model(input=vgg16.input, output=top_model(vgg16.output))

# 最後のconv層の直前までの層をfreeze
for layer in model.layers[:15]:
    layer.trainable = False

# Fine-tuningのときはSGDの方がよい
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
```

In [1]:
from keras import regularizers, Input
from keras.applications.vgg16 import VGG16
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.models import Sequential, Model
from keras.models import model_from_json, load_model
from keras.optimizers import Adam, Adagrad, SGD
from keras.callbacks import EarlyStopping, TensorBoard
from keras.utils import plot_model

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import copy
import datetime
import glob
import numpy as np
import os
import pandas as pd

from FX.FX.core import utils
# from FX.FX import KerasModelAdapter

Using TensorFlow backend.


## データセットの読み込み

In [2]:
# basepath = "C:/Users/Surpris/Desktop/20170918/"
basepath = "../../images/20170918/"
filelist = np.array(glob.glob(os.path.join(basepath, "images-ohlc/*.png")))
data = pd.read_csv(basepath + "FXTF/USDJPY-cd1_20170806_k030.csv")
y = data[["label1", "label2", "label3"]].as_matrix()[9:].copy()

Xpath_train, Xpath_test, y_train, y_test = train_test_split(filelist, y, test_size=0.3)

In [3]:
grpX, grpY = utils.grouping_dataset(Xpath_train, y_train, 16)

## モデルの構築

### VGG16モデルの読み込み
[Kerasの公式サイト（日本語版）](https://keras.io/ja/applications/#vgg16)にもあるが、VGG16は幅と高さが４８以上、チャネルが３つのデータを入れる必要がある。   
つまり $(h, w, c) = (\geq 48, \geq 48, 3)$ でなくてはならない。   

モデルが存在しない場合は最初に学習済みモデル（HDF5形式）のダウンロードが始まる。   
inputのサイズが(100, 100, 3)の時にファイルサイズが56.1 MBであった。それほど大きくはない。   
モデルのダウンロード先は`%USERPROFILE%\.keras\models\`フォルダである。

In [5]:
X_test = utils.load_images_from_filelist(Xpath_test[:2], channel=3)

In [6]:
try:
    img1
except:
    img1 = X_test[0]
input_shape = img1.shape
input_tensor = Input(shape=input_shape)

vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

### 全結合層の構築と接続

In [7]:
# 構築
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(y.shape[1], activation='softmax'))

# 接続
model = Model(inputs=vgg16.input, outputs=top_model(vgg16.output))

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0         
__________

### fine-tuningしたい層以外をfreeze
fine-tuningしたいのは全結合層とその手前の畳み込み層とする。  
次の図のような感じ。   
![](../../images/VGG16_image.webp)   
（[Deep learningで画像認識⑧〜Kerasで畳み込みニューラルネットワーク vol.4〜 - IMACEL Academy -人工知能・画像解析の技術応用に向けて-|LPixel(エルピクセル)](https://lp-tech.net/articles/ks8F9)より引用）

In [8]:
# 最後のconv層の直前までの層をfreeze
for layer in model.layers[:15]:
    layer.trainable = False

# Fine-tuningのときはSGDの方がよい（理由は不明）
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

## 訓練

メモリが足りないので、セルフバッチ学習を行ってみる。

In [ ]:
callbacks = []
callbacks.append(EarlyStopping(monitor='val_loss', patience=1))

X_fpath, ys = grpX[0], grpY[0]
N = len(X_fpath)
batch_size = 100
for ii in range(N//batch_size):
    start = ii * batch_size
    end = (ii+1) * batch_size
    X = utils.load_images_from_filelist(X_fpath[start:end], 3)
    y = ys[start:end]
    hist = model.fit(X, y, batch_size=batch_size, epochs=10,
                     validation_split=0.1, 
                     callbacks=callbacks, verbose=0)
    if ii == 5:
        break

In [24]:
hist.history

{'acc': [0.4444444477558136,
  0.42222222685813904,
  0.43333333730697632,
  0.47777777910232544,
  0.4444444477558136,
  0.42222222685813904,
  0.47777777910232544,
  0.4444444477558136,
  0.56666666269302368],
 'loss': [0.99315696954727173,
  1.0379666090011597,
  1.0105534791946411,
  1.0045055150985718,
  0.99449193477630615,
  1.016797661781311,
  0.97113806009292603,
  1.0045349597930908,
  0.90805554389953613],
 'val_acc': [0.5,
  0.5,
  0.40000000596046448,
  0.40000000596046448,
  0.40000000596046448,
  0.40000000596046448,
  0.40000000596046448,
  0.5,
  0.5],
 'val_loss': [0.83791673183441162,
  0.83280009031295776,
  0.82866752147674561,
  0.82555615901947021,
  0.82388228178024292,
  0.82312887907028198,
  0.82300138473510742,
  0.82352769374847412,
  0.82465255260467529]}

In [ ]:
model.save("../../images/20170918/ML/model_VGG16_2.h5")

## 予測

In [ ]:
model = load_model("../../images/20170918/ML/model_VGG16_2.h5")
X_test = utils.load_images_from_filelist(Xpath_test[:6000], channel=3)

In [ ]:
utils.plot_probability(model, X_test)

In [ ]:
utils.calc_accuracy_above_threshold(model, X_test, y_test[:len(X_test)], threshold=0.60, verbose=1)

## まとめ
VGG16を用いた転移学習をFXTFのOHLC図に適用した。   
精度が上がったような雰囲気はないことから、いまのOHLC図ではうまく予測ができないのかもしれない。   
色を付けてみるとか、Volumeを加えてみるとか試すのもよいか？   
もしくは、単に学習回数などが足りないか。   
Batch normalizationを加えたときのように予測確率の変な飛びがないようだから、期待される動作はしていると思われる。

VGG16を利用する際に気を付けることとして、訓練時にメモリを数GBは利用するという点か。   
訓練用データを一度に多く与えるとメモリ使用量が増加するため、少しずつ与えて訓練するのがよいかもしれない。